In [8]:
# 環境設置
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys  # 鍵盤操作
from selenium.common.exceptions import NoSuchElementException, TimeoutException
# ---------- 等待頁面加載完成
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
# ----------
import pandas as pd
import pandas.io.formats.excel
import requests
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()
from urllib.request import HTTPError
from bs4 import BeautifulSoup
import time

def main():
    # 設定 driver
    global my_options
    global capabilities
    my_options = Options()
    my_options.add_argument("--incognito")  # 開啟無痕模式
    # my_options.add_argument("--headless")  # 不開啟實體瀏覽器
    capabilities = DesiredCapabilities.CHROME
    capabilities["pageLoadStrategy"] = "none"

In [9]:
if __name__ == '__main__':
    main()
    driver = webdriver.Chrome(options=my_options, desired_capabilities=capabilities)
    wait = WebDriverWait(driver, 10)

In [ ]:
# so-so version ~
# webpac_gov_crawler
def click_more_btn(driver):
    try:
        while True:
            more_btn = wait.until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, "a.btnstyle.bluebg3.morewidth"))
            )
            if more_btn:
                time.sleep(2.5)  # 不得已的強制等待
                more_btn.click()
                # driver.find_element_by_css_selector("a.btnstyle.bluebg3.morewidth").click()
    except:
        time.sleep(2.5)

# 宜蘭縣公共圖書館、桃園市立圖書館、高雄市立圖書館、屏東縣公共圖書館、花蓮縣圖書館、澎湖縣圖書館
def webpac_gov_crawler(org, url, ISBN, driver, wait):
    try:
        # 進入＂搜尋結果＂頁面
        search_url = url + ISBN
        driver.get(search_url)

        # 計算＂搜尋結果的數量＂
        book_list = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.booklist"))
        )
        results = len(book_list)
        print(f"搜尋結果數量有：{results}筆", end="    ")

        time.sleep(2)  # 頁面加載，用強制等待

        # 依照搜尋結果的數量分成：一筆和多筆兩種情形
        if results < 2:
            if wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.bookplace_list"))):
                click_more_btn(driver)
                return pd.read_html(driver.page_source, encoding="utf-8")[-1]
        else:
            soup = BeautifulSoup(driver.page_source, "html.parser")  # 用以當搜尋結果多筆時，找尋不同＂詳細書目＂的網址
            
            tgt = []
            url = url.split("search?")[0]  # 取得＂搜尋主頁＂頁面
            for li in soup.find_all("div", "booklist"):
                href = url + li.find("h2").find("a")["href"]
                
                driver.get(href)
                if wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.bookplace_list"))):
                    click_more_btn(driver)
                tgt.append(pd.read_html(driver.page_source, encoding="utf-8")[-1])
            return pd.concat(tgt, axis=0, ignore_index=True)
    except:
        print(f"「{url}」無法爬取！")

In [10]:
# perfect version!!!
# 國立宜蘭大學、佛光大學、嘉南藥理大學
def webpac_jsp_crawler(org, url, ISBN, driver, wait):
    try:
        # 進入＂搜尋主頁＂
        driver.get(url)

        # 定位＂下拉式選單＂，選擇 ISBN
        search_field = wait.until(
            EC.presence_of_element_located((By.NAME, "search_field"))
        )
        select = Select(search_field)
        select.select_by_value("ISBN")

        # 定位＂搜尋欄＂，輸入 ISBN
        search_input = wait.until(
            EC.presence_of_element_located((By.NAME, "search_input"))
        )
        search_input.send_keys(ISBN)
        search_input.submit()

        # 定位＂表格＂，爬取內容
        if wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.order"))):
            return pd.read_html(driver.page_source, encoding="utf-8")[-2]
    except:
        print(f"「{url}」無法爬取！")

In [ ]:
# 國立臺灣海洋大學、國立臺灣師範大學、中央研究院、國立陽明大學、國立中央大學、中原大學
# 國立臺灣科技大學、中國文化大學、輔仁大學
def easy_crawler(position, org, url, ISBN):
    try:
        # 組合成書的網址
        url = url + ISBN
        # 載入 html，如果發生 HTTPError，那麼就使用 requests.get(url, verify=False)
        try:
            dfs = pd.read_html(url, encoding="utf-8")
        except HTTPError:
            resp = requests.get(url, verify=False)  # 設定 verify=False，以解決 SSLError
            dfs = pd.read_html(resp.text, encoding="utf-8")
        # 定位表格
        tgt = dfs[position]
        
        # tgt.insert(0, "連結", [url for i in range(tgt.shape[0])])
        # tgt.insert(0, "圖書館", [org for i in range(tgt.shape[0])])
        return tgt
    except:
        print(f"「{url}」無法爬取！")

In [ ]:
def klccab_crawler(org, ISBN, driver, wait):
    try:
        url = f"https://webpac.klccab.gov.tw/webpac/search.cfm?m=as&k0={ISBN}&t0=i&c0=and&y10=&y20=&cat0=&dt0=&l0=&lv0=&lc0="
        driver.get(url)

        time.sleep(5)  # 基隆的系統太詭異了，強制等待
        soup = BeautifulSoup(driver.page_source, "html.parser")
        results = len(soup.find_all("div", "list_box"))
        if results < 2:
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.list.list_border")))
            tgt = pd.read_html(driver.page_source)[0]
        else:
            tgt = []
            for li in soup.find_all("div", "list_box"):
                url_temp = "https://webpac.klccab.gov.tw/webpac/" + li.find("a", "btn")["href"]
                driver.get(url_temp)
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.list.list_border")))
                tgt.append(pd.read_html(driver.page_source, encoding="utf-8")[0])
            tgt = pd.concat(tgt, axis=0, ignore_index=True)
        return tgt
    except:
        print(f"「{url}」無法爬取！")

In [ ]:
def tpml_crawler(org, url, ISBN, driver, wait):
    # 進入《館藏系統》頁面
    driver.get(url)
    
    # 等待＜下拉式選單＞元素出現
    wait.until(EC.presence_of_element_located((By.NAME, "search_field")))

    # 定位＜下拉式選單＞，以 ISBN 搜尋方式搜尋
    select = Select(driver.find_element_by_name("search_field"))
    select.select_by_visible_text(u"ISBN")
    search_input = driver.find_element_by_name("search_input")
    search_input.send_keys(ISBN)
    search_input.submit()
    
    # 等待＜表格＞出現
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.order")))
    
    # 取得當前網頁的 html 進行解析，以取得 DataFrame
    html_text = driver.page_source
    
    dfs = pd.read_html(html_text, encoding="utf-8")
    tgt = dfs[19]
    return tgt

In [3]:
def ntu_crawler(ISBN, driver):
    df_lst = []
    
    url = "https://ntu.primo.exlibrisgroup.com/discovery/search?query=any,contains," + ISBN + "&tab=Everything&search_scope=MyInst_and_CI&vid=886NTU_INST:886NTU_INST&offset=0"
    driver.get(url)
    time.sleep(5)
    title = driver.find_element_by_class_name('item-title').click()

    time.sleep(3)
    try:
        try:
            back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
        except:
            back = None
        if back != None:
            back.click()

        thelist = driver.find_elements_by_class_name("layout-align-space-between-center.layout-row.flex-100")
        for row in thelist:
            plist = row.find_elements_by_tag_name("p")
            where = row.find_elements_by_tag_name("h3")
            i = len(where)
            for sth in plist:
                a = sth.find_elements_by_tag_name("span")
                for _ in range(i): 
                    new_row = ['台灣大學', where[_].text, a[4].text, a[0].text, url]
                    df_lst.append(new_row)
                    break
                break
    except:
        return pd.DataFrame(df_lst)

In [ ]:
def 國家圖書館(url, ISBN, driver):
    try:
        # 進入＂搜尋主頁＂
        driver.get(url)

        # 定位＂下拉式選單＂，選擇 ISBN
        select = Select(driver.find_element_by_name("find_code"))
        select.select_by_value("ISBN")
        search_input = driver.find_element_by_name("request")
        search_input.send_keys(ISBN)
        search_input.submit()

        # 點擊＂書在哪裡(請點選)＂
        driver.find_element_by_xpath('/html/body/table[10]/tbody/tr/td[1]/table/tbody/tr[1]/td[2]/a').click()

        # 定位＂表格＂，爬取內容
        tgt = pd.read_html(driver.page_source)[-2]  # 會有 NaN，要加 .fillna("")？

        # 清理、整理資料
        tgt['連結'], tgt['圖書館'] = driver.current_url, '國家圖書館'
        tgt = tgt[['圖書館', '分館/專室', '索書號', '狀態/調閱日期', '連結']]  # 篩選 column label
        tgt.set_axis(['圖書館', '館藏地', '索書號','館藏狀態', '連結'], axis='columns', inplace=True)  # 重新命名 column label
        return tgt
    except:
        print(f"「{url}」無法爬取！")